<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/BigDL_LLM_cpu_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 安装适应的包

In [1]:
!pip install -q datasets tiktoken
!pip install --pre --upgrade bigdl-llm[all] # install bigdl-llm with 'all' option
!pip install transformers==4.37.0  # install the transformers which support Qwen2

  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0
  Using cached transformers-4.37.0-py3-none-any.whl (8.4 MB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstal

## 在Colab中需要运行以下代码

In [2]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 1019, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 1019 (delta 290), reused 315 (delta 247), pack-reused 643
Receiving objects: 100% (1019/1019), 3.60 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (722/722), done.
/content/Zero-Haruhi


## 使用RAG进行增强，经过测试会有很长的推理时间，推荐注释掉chatbot.add_rag_prompt_after_persona()

In [3]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_qwen_base import get_response

chatbot = ChatHaruhi( role_name = 'haruhi', llm = get_response)
chatbot.add_rag_prompt_after_persona()
prompt_print = chatbot.get_message(user = "", text = "老师: 阿虚去哪儿了！")

query_rags: [{'n': 5, 'max_token': 1000, 'query': '老师: 阿虚去哪儿了！', 'lid': 8}] rest_limit = 1503


In [4]:
model_path = "silk-road/Haruhi-Zero-7B-0_4"

In [5]:
#
# Copyright 2016 The BigDL Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import torch
import time
# import argparse
import numpy as np

from transformers import AutoTokenizer
from bigdl.llm.transformers import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True, trust_remote_code=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# prompt = args.prompt

# Generate predicted tokens
with torch.inference_mode():
    messages = prompt_print
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
        )
    model_inputs = tokenizer([text], return_tensors="pt")
    st = time.time()
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=32
        )
    end = time.time()
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f'Inference time: {end-st} s')
    print('-'*20, 'Prompt', '-'*20)
    # print(prompt)
    print('-'*20, 'Output', '-'*20)
    print(response)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Inference time: 697.6596043109894 s
-------------------- Prompt --------------------
-------------------- Output --------------------
春日:「阿虚，你去哪儿了？怎么突然不见了？」
